# builtins module: the float class

The ```float``` class is an abbreviation for a floating point number and is Pythons default representation for a number that has a non-integer component. The floating point number is displayed in the decimal numbering system using ten characters ```0```, ```1```, ```2```, ```3```, ```4```, ```5```, ```6```, ```7```, ```8```, ```9``` however under the hood is physically stored using bits which have the combination ```0``` and ```1```. 

8 bytes (64 bits) are used to store the number using the IEEE-754 convention. Recall that a bit can essentially be conceptualised as a binary switch and so a ```float``` under the hood can look like the following:

<img src='./images/img_001.png' alt='img_001' width='600'/>

When 0 and 1 are used for the two switch positions this becomes:

```
00111111 10111001 10011001 10011001 10011001 10011001 10011001 10011010
```

However the arrangement of the bits is encoded in the following form sign (1), exponent (11) and fraction (53):

```
0 01111111011 1001100110011001100110011001100110011001100110011010
```

Notice the recursion in the fraction; this number repeats forever but is ultimately truncated to fit the 53 bits. This kind of behaviour is very common with dealing with floating point numbers creating subtle rounding issues.

## Initialisation Signature

The ```float``` is a fundamental datatype and normally instantiated using the number with the decimal point (fixed format):

In [1]:
3.14

3.14

For very small and very large numbers an exponent to the base ```10``` (scientific notation) is normally used, the radius of a hydrogen atom, radius of a human and radius of the sun are for example:

In [2]:
rhydrogen = 1.2e-10

In [3]:
rhydrogen

1.2e-10

In [4]:
rhuman = 1

In [5]:
rhuman

1

In [6]:
rsun = 6.957e8

In [7]:
rsun

695700000.0

The initialisation signature of the ```float``` class is typically only used for casting:

In [8]:
float?

Init signature: float(x=0, /)
Docstring:      Convert a string or number to a floating point number, if possible.
Type:           type
Subclasses:     

It has an input argument ```x``` which has a default value of ```0``` returning the ```float``` instance ```0.0```, notice the inclusion fo the decimal point:

In [9]:
float()

0.0

In [10]:
0

0

The initialisation signature can be used to cast an ```int``` to a ```float``` or a ```str``` to a ```float```:

In [11]:
float(1)

1.0

In [12]:
float('3.14')

3.14

## Format Specifiers

To understand the difference in the numbers above a formatted string can be used with a fixed format specifier:

```python
{num:0w.pf}
```

In format specifier: 
* 0 is an instruction to show leading zeros.
* w is the character width of the number in the formatted string and includes the decimal point.
* p is the precision, i.e. the number of digits past the decimal point.
* f is used to represent the fixed format.

If for example the three radii explored above are shown with 20 digits before and after the decimal point, the difference in magnitude between the very small radius and the very large radius can be seen:

In [13]:
f'{rhydrogen:041.20f}'

'00000000000000000000.00000000012000000000'

In [14]:
f'{rhuman:041.15f}'

'0000000000000000000000001.000000000000000'

In [15]:
f'{rsun:041.15f}'

'0000000000000000695700000.000000000000000'

In the physical world, every number measured has an associated error. The sun for example is constituted of an exceedingly large number of hydrogen atoms:

In [16]:
numhydrogen = rsun / rhydrogen 

In [17]:
numhydrogen

5.7975e+18

In [18]:
f'{numhydrogen:041.15f}'

'0000005797500000000000000.000000000000000'

Because this is such a large number... its associated error is likely to be larger than the width of more than a million, million hydrogen atoms. Naturally the error in the separate measurement involving the width of a single hydrogen atom is much smaller.

Quoting the numbers to the above number of zeros doesn't physically make sense as they are not measured to such accuracy and it is not human readable to read that many zeros. Instead the format specifier can be changed to ```e``` to give:

In format specifier: 
* 0 is an instruction to show leading zeros (unspecified).
* w is the character width of the number in the formatted string and includes the decimal point (unspecified).
* p is the precision, i.e. the number of digits past the decimal point.
* e is used to represent the exponent (scientific notation).

In [19]:
f'{rsun:.3e}'

'6.957e+08'

In [20]:
f'{rhydrogen:.3e}'

'1.200e-10'

If the division is again examined:

In [21]:
f'{rsun / rhydrogen:.3e}'

'5.798e+18'

Notice that the mantissa of the first number is divided by the second number:

In [22]:
6.957 / 1.200

5.7975

And there is a subtraction of exponents, the exponent of the first number subtracts the exponent of the second number:

In [23]:
8 - (-10)

18

Lets examine the inverse operation multiplication:

In [24]:
f'{rhydrogen:.3e}'

'1.200e-10'

In [25]:
f'{numhydrogen:.3e}'

'5.798e+18'

If multiplication is examined:

In [26]:
f'{rhydrogen * numhydrogen:.3e}'

'6.957e+08'

The mantissa of the two numbers are multiplied:

In [27]:
1.200 * 5.798

6.9576

And the exponents are added:

In [28]:
-10 + 18

8

Operations involving addition or subtraction of the small number from the large number result in the large  number being unchanged as the small number is orders of magnitude smaller than the larger numbers error.

## Binary Encoding

Although a floating point number is displayed in decimal:

In [29]:
0.1

0.1

Under the hood a floating point number is encoded in binary using the IEEE754 technical standard for floating point numbers. The pickle module is used for serialisation of a Python object:

In [30]:
import pickle

The ```pickle.dumps``` function can be used to dump an ```object``` to a ```bytes``` string:

In [31]:
pickle.dumps(0.1)

b'\x80\x04\x95\n\x00\x00\x00\x00\x00\x00\x00G?\xb9\x99\x99\x99\x99\x99\x9a.'

This can be viewed as a hexadecimal string:

In [32]:
pickle.dumps(0.1).hex()

'8004950a00000000000000473fb999999999999a2e'

Pickle prefixes and suffixes information to the ```bytes``` string:

|Hex Value|Meaning|
|---|---|
|80|Start OptCode: pickle.PROTO.hex()|
|04|Protocol Version: pickle.DEFAULT_PROTOCOL|
|95|Frame OptCode: pickle.FRAME.hex()|
|0a 00 00 00 00 00 00 00|Frame Size: 8 bytes little endian|
|47|pickle.BINFLOAT|
|3f b9 99 99 99 99 99 9a|0.1 in 64 Bit IEEE|
|2e|Stop OptCode: pickle.STOP.hex()|

The 64 bits (16 hexadecimal character) of interest can be indexed:

In [33]:
pickle.dumps(0.1).hex()[24:40]

'3fb999999999999a'

This can then be cast back into an integer so it can be shown in binary:

In [34]:
bin(int('0x' + '3fb999999999999a', base=16)).removeprefix('0b').zfill(64)

'0011111110111001100110011001100110011001100110011001100110011010'

<img src='./images/img_001.png' alt='img_001' width='600'/>

In IEEE754 there is:
* 1 bit for the sign
* 11 bits for the biased exponent
* 52 bits for the fraction

In [35]:
(bin(int('0x' + '3fb999999999999a', base=16)).removeprefix('0b').zfill(64)[0],
bin(int('0x' + '3fb999999999999a', base=16)).removeprefix('0b').zfill(64)[1:12],
bin(int('0x' + '3fb999999999999a', base=16)).removeprefix('0b').zfill(64)[12:])

('0', '01111111011', '1001100110011001100110011001100110011001100110011010')

<img src='./images/img_002.png' alt='img_002' width='600'/>

The sign is ```0``` for positive numbers and ```1``` for negative numbers.

The biased exponent is biased by ```1023```:

In [36]:
bin(1023)

'0b1111111111'

The unbiased exponent is therefore:

In [37]:
0b01111111011 - 0b1111111111

-4

Every number expressed in binary scientific notation will begin with ```1.``` as the first non-zero value is placed before the binary point (base 2) and therefore the only possible non-zero value is ```1```. This ```1.``` is not encoded in order to conserve memory. Putting the above together:

In [38]:
('+', '-4', '1.1001100110011001100110011001100110011001100110011010')

('+', '-4', '1.1001100110011001100110011001100110011001100110011010')

Using the ```-4``` to shift the fraction ```4``` decimal places and then adding the sign gives the number as a binary float:

In [39]:
'+0.00011001100110011001100110011001100110011001100110011010'

'+0.00011001100110011001100110011001100110011001100110011010'

So the relatively straightforward number ```0.1``` in base 10 (digits 0, 1, 2, 3, 4, 5, 6, 7, 8, 9):

```python
+0.1
```

Becomes the following number in base 2 (digits 0, 1):

```python
+0.00011001100110011001100110011001100110011001100110011010
```

For convenience the above will be implemented in a function:

In [40]:
def float2binaryfloat(f1):
    hex1 = pickle.dumps(f1).hex()[24:40]
    sign1 = bin(int('0x' + hex1, base=16)).removeprefix('0b').zfill(64)[0]
    biasedexponent = bin(int('0x' + hex1, base=16)).removeprefix('0b').zfill(64)[1:12]
    mantissa = bin(int('0x' + hex1, base=16)).removeprefix('0b').zfill(64)[12:]

    if sign1 == '0':
        sign2 = '+'
    else:
        sign2 = '-'

    exponent2 = int(biasedexponent, base=2) - 1023
    
    if exponent2 < 1:
        absolute = '0.' + '0' * abs(exponent2 + 1) + '1' + mantissa
        result = sign2 + absolute
    elif exponent2 > 1:
        mantissa2 = '1' + mantissa
        absolute = mantissa2[:exponent2+1] + '.' + mantissa2[exponent2+1:]
        result = sign2 + absolute
    else:
        mantissa2 = '1.' + mantissa
        result = sign2 + mantissa2
    return result

This can be used to examine other ```float``` instances:

In [42]:
float2binaryfloat(0.125)

'+0.0010000000000000000000000000000000000000000000000000000'

In [44]:
float2binaryfloat(0.25)

'+0.010000000000000000000000000000000000000000000000000000'

In [46]:
float2binaryfloat(0.5)

'+0.10000000000000000000000000000000000000000000000000000'

In [48]:
float2binaryfloat(1.0)

'+0.010000000000000000000000000000000000000000000000000000'

In [50]:
float2binaryfloat(2.0)

'+1.0000000000000000000000000000000000000000000000000000'

In [52]:
float2binaryfloat(4.0)

'+100.00000000000000000000000000000000000000000000000000'

In [54]:
float2binaryfloat(8.0)

'+1000.0000000000000000000000000000000000000000000000000'

Notice that these numbers are all multiples of the power of ```2``` and therefore all encode completely in binary and have a large number of trailing zeros:

In [63]:
for num in range(-3, 4):
    print(2 ** num)

0.125
0.25
0.5
1
2
4
8


 If other floating point numbers are examined, recursion is observed:

In [55]:
float2binaryfloat(0.1)

'+0.00011001100110011001100110011001100110011001100110011010'

In [56]:
float2binaryfloat(-0.1)

'-0.00011001100110011001100110011001100110011001100110011010'

In [57]:
float2binaryfloat(0.2)

'+0.0011001100110011001100110011001100110011001100110011010'

In [58]:
float2binaryfloat(0.3)

'+0.010011001100110011001100110011001100110011001100110011'

In [59]:
float2binaryfloat(123.4)

'+1111011.0110011001100110011001100110011001100110011010'

Since these numbers are stored to a limited fixed precision the number must be truncated at the last bit and there is therefore an associated recursive rounding error. This phenomenon can be seen when working with floating point numbers: 

In [64]:
0.1 + 0.2

0.30000000000000004

In [65]:
0.3

0.3

And care has to therefore be taken when using comparison operators:

In [66]:
0.1 + 0.2 == 0.3

False

This is because the value calculated on the left hand side has a rounding error and is not precisely the value on the right hand side.

If the ```float``` instance ```0.1``` is viewed as a hexadecimal:

In [67]:
pickle.dumps(0.1).hex()[24:40]

'3fb999999999999a'

Recall in IEEE754 there is:

* 1 bit for the sign
* 11 bits for the biased exponent
* 52 bits for the fraction


This means the first 12 bits, is the sign and exponent, which is the first 3 hexadecimal characters (12 / 4):

In [68]:
signbiasexponent = pickle.dumps(0.1).hex()[24:27]
signbiasexponent

'3fb'

The remaining 52 bits (52 / 4), 13 hexadecimal characters are for the fraction:

In [69]:
fraction = pickle.dumps(0.1).hex()[27:40]
fraction

'999999999999a'

The signexponent can be converted to binary and examined seperately:

In [70]:
bin(int(signbiasexponent, base=16)).removeprefix('0b').zfill(12)

'001111111011'

Recall that a sign of 0 is + and a sign of 1 is - :

In [71]:
sign = bin(int(signbiasexponent, base=16)).removeprefix('0b').zfill(12)[0]
sign

'0'

And that the biased exponent is:

In [72]:
bin(int(signbiasexponent, base=16)).removeprefix('0b').zfill(12)[1:]

'01111111011'

With bias:

In [73]:
bin(int(1023)).removeprefix('0b').zfill(11)

'01111111111'

The unbiased component is therefore:

In [74]:
0b01111111011 - 0b01111111111

-4

The three components of the ```float```:

* SIGN: +
* EXPONENT: -4
* MANTISSA: 99999999999a

Can be expressed using the floating point hex format:

```python
'SIGN0x1.MANTISSApEXPONENT'
```

Where ```0x``` means hexadecimal and ```p``` means power.

For the number above this is:

In [75]:
'+0x1.999999999999ap-4'

'+0x1.999999999999ap-4'

The ```float``` class has a method ```hex``` which returns this value:

In [76]:
num = 0.1

In [77]:
num.hex()

'0x1.999999999999ap-4'

There is an associated class method ```fromhex``` which is an alternative constructor and is used to construct a ```float``` instance from a hexadecimal string:

In [78]:
float.fromhex('-0x1.999999999999ap-4')

-0.1

## Identifiers

The custom function ```print_identifier_group``` can be imported: 

In [79]:
from helper_module import print_identifier_group

And can be used to examine the identifiers in both the ```int``` and ```float``` classes. Notice that the vast majority of identifiers are consistent for both classes because they have consistent behaviour:

In [84]:
print_identifier_group(int, kind='all', second=float, show_only_intersection_identifiers=True)

datamodel attribute: ['__doc__']
datamodel method: ['__abs__', '__add__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__int__', '__le__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__pos__', '__pow__', '__radd__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmod__', '__rmul__', '__round__', '__rpow__', '__rsub__', '__rtruediv__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__']
constant: []
attribute: ['imag', 'real']
method/function: ['as_integer_ratio', 'conjugate', 'is_integer']
upper class: []
lower class: []


The ```float``` class lacks byte related methods and datamodel methods and fraction related attributes:

In [85]:
print_identifier_group(int, kind='all', second=float, show_unique_identifiers=True)

datamodel attribute: []
datamodel method: ['__and__', '__index__', '__invert__', '__lshift__', '__or__', '__rand__', '__rlshift__', '__ror__', '__rrshift__', '__rshift__', '__rxor__', '__xor__']
constant: []
attribute: ['denominator', 'numerator']
method/function: ['bit_count', 'bit_length', 'from_bytes', 'to_bytes']
upper class: []
lower class: []


The additional 2 methods are the ```hex``` and ```fromhex``` methods which were examined earlier and are used to represent or construct a ```float``` for a hexadecimal float ```str```. The additional datamodel method ```__getformat__``` (*dunder getformat*) is not typically used outwith Pythons test suites:

In [83]:
print_identifier_group(float, kind='all', second=int, show_unique_identifiers=True)

datamodel attribute: []
datamodel method: ['__getformat__']
constant: []
attribute: []
method/function: ['fromhex', 'hex']
upper class: []
lower class: []


The ```float``` class has the commonly used datamodel methods inherited from ```object``` (most of these are redefined in the ```float``` class):

|Data Model Identifier|Function or Class|Description|
|---|---|---|
|\_\_class\_\_|builtins.type|class type|
|\_\_new\_\_|cls|constructor|
|\_\_init\_\_|cls?|initialisation signature|
|\_\_dir\_\_|builtins.dir|directory|
|\_\_str\_\_|builtins.str|informal string representation|
|\_\_repr\_\_|builtins.repr|formal string representation|
|\_\_format\_\_|builtins.str.format|format spec for {}|
|\_\_sizeof\_\_|sys.getsizeof|get size of|
|\_\_hash\_\_|builtins.hash|hash value (immutable)|
|\_\_getattribute\_\_|builtins.getattr or inst.attr|get attribute|
|\_\_setattr\_\_|builtins.setattr or inst.attr = val|set attribute (mutable)|
|\_\_delattr\_\_|builtins.delattr or del inst.attr|delete attribute (mutable)|
|\_\_eq\_\_|==|is equal to|
|\_\_ne\_\_|!=|not equal to|
|\_\_gt\_\_|>|greater than (ordinal)|
|\_\_ge\_\_|>=|greater than or equal to (ordinal)|
|\_\_lt\_\_|<|less than to (ordinal)|
|\_\_le\_\_|<=|less than or equal to (ordinal)|

And the datamodel identifiers that are used by the pickle module or for subclassing:

|Data Model Identifier|Function or Class|Description|
|---|---|---|
|\_\_getstate\_\_| |Helper for pickle|
|\_\_reduce\_\_| |Helper for pickle|
|\_\_reduce_ex\_\_| |Helper for pickle|
|\_\_init_subclass\_\_| |Called when Subclassed|
|\_\_subclasshook\_\_|builtins.issubclass|Abstract Classes can Override this|

The ```float``` class also has the numeric datamodel identifiers:

|Data Model Identifier|Function or Class|Description|
|---|---|---|
|\_\_bool\_\_|builtins.bool|unitary cast to boolean|
|\_\_int\_\_|builtins.float|unitary cast to integer|
|\_\_float\_\_|builtins.float|unitary cast to floating point number|
|\_\_pos\_\_|+|unitary positive|
|\_\_neg\_\_|-|unitary negative|
|\_\_abs\_\_|builtins.abs|unitary absolute value|
|\_\_round\_\_|builtins.round|unitary round to nearest integer|
|\_\_floor\_\_|math.floor|unitary floor integer|
|\_\_ceil\_\_|math.ceil|unitary ceiling integer|
|\_\_trunc\_\_|math.trunc|unitary truncate to integer|
|\_\_add\_\_|+|binary add|
|\_\_sub\_\_|-|binary subtract|
|\_\_mul\_\_|*|binary multiply|
|\_\_pow\_\_|**|binary raise to power of|
|\_\_floordiv\_\_|//|binary integer division value|
|\_\_mod\_\_|%|binary integer division modulus|
|\_\_divmod\_\_|builtins.divmod|binary integer division (value, modulus)|
|\_\_truediv\_\_|/|binary float division|
|\_\_getnewargs\_\_| |helper function for pickle|  
  
The 7 binary operators have a reverse equivalent, for example multiplication ```__mul__``` has reverse multiplication ```__rmul__```. The former computes ```self * value``` and the latter computes ```value * self```.

## Type Casting Datamodel Identifiers

The ```int```, ```bool``` and ```float``` classes will take a ```float``` instance and return a cast instance. For the ```int``` the ```float``` is truncated:

In [87]:
int(3.14)

3

For a ```bool```, only ```0.0``` is cast to ```False``` and any non-zero ```float``` instance is cast to ```True```:

In [88]:
bool(3.14)

True

In [89]:
bool(0.0)

False

In [90]:
bool(-0.1)

True

Casting a ```float``` instance to a ```float``` instance leaves it unchanged.

In [91]:
float(3.14)

3.14

## Rounding Datamodel Identifiers

The ```float``` class has the following four datamodel methods ```__round__```, ```__floor__```, ```__ceil__``` and ```__trunc__``` which all round a ```float``` instance into an ```int``` instance. The latter 3 all map to functions found in the ```math``` module. Therefore in order to use these functions the ```math``` module has to be imported:

In [92]:
import math

The floating point number ```tau``` can be rounded:

In [93]:
from math import tau

In [94]:
tau

6.283185307179586

```math.floor``` returns the floor integer, that is the first ```int``` instance below the ```float``` instance:

In [95]:
math.floor(tau)

6

In [96]:
math.floor(-tau)

-7

```math.ceil``` returns the ceiling integer, that is the first ```int``` instance above the ```float``` instance:

In [97]:
math.ceil(tau)

7

In [98]:
math.ceil(-tau)

-6

```math.trunc``` truncates the ```float``` instance ignoring anything past the decimal point. It effectively gives the same result as casting a ```float``` to an ```int``` using the ```int``` class: 

In [99]:
math.trunc(tau)

6

In [100]:
math.trunc(-tau)

-6

The ```round``` function will round a ```float``` down to the nearest ```int``` instance by default:

In [101]:
round(tau)

6

The ```round``` function has the keyword input argument ```ndigits``` which can instead be used to return a ```float``` instance with a specified number of digits past the decimal point:

In [102]:
round?

Signature: round(number, ndigits=None)
Docstring:
Round a number to a given precision in decimal digits.

The return value is an integer if ndigits is omitted or None.  Otherwise
the return value has the same type as the number.  ndigits may be negative.
Type:      builtin_function_or_method

In [103]:
round(tau, 3)

6.283

## Unitary Operators

The positive ```+``` and negative operators behave consistently with their counterparts in the ```int``` class but return a ```float``` instance instead of an ```int```:

In [104]:
+tau

6.283185307179586

In [105]:
-tau

-6.283185307179586

The absolute function also behaves consistently between the ```int``` and ```float``` class but returns an ```int``` or ```float``` respectively:

In [106]:
abs(-tau)

6.283185307179586

## Binary Operators

The binary operators behave consistently to their counterparts in the ```int``` class returning a ```float``` instead of an ```int```. Recursive rounding errors however are commonly encountered:

In [107]:
0.1 + 0.2

0.30000000000000004

In [108]:
0.3 - 0.1

0.19999999999999998

In [109]:
0.1 * 3.14

0.31400000000000006

In [110]:
3.14 ** 0.5

1.772004514666935

In [111]:
3.14 / 0.21

14.952380952380954

## Comparison Operators

The 6 comparison operators are setup to be consistent with their counterpart in the ```int``` class. However they are stricter than the recursive rounding errors encountered with the ```float``` class and may lead to unexpected results. Generally it is recommended to avoid their use with flaoting point numbers or to use the ```round``` function to limit the precision of the floating point numbers:

In [112]:
0.1 + 0.2 == 0.3

False

In [113]:
round(0.1 + 0.2, ndigits=6) == round(0.3, ndigits=6)

True

## Integer Based Identifiers

The method ```is_integer``` will check to see if a float is an integer. Recall this method requires use of ```,``` indexing from an instance name and won't work with a floating point number directly due to confusion with the decimal point:

In [114]:
two = 2.0

In [115]:
two.is_integer?

Signature: two.is_integer()
Docstring: Return True if the float is an integer.
Type:      builtin_function_or_method

In [116]:
two.is_integer()

True

In [117]:
tau.is_integer()

False

The integer ratio method can be used to obtain a 2 element tuple where the first element is the ```int``` numerator and the second element is the ```int``` denominator. This method works consistently with the integer equivalent if ```float.is_integer``` returns ```True```. When this is ```False```, care should be taken as a very large numerator and denominator is often obtained from ```float``` instances and the numbers obtained are also distorted by rounding errors:

In [118]:
two.as_integer_ratio?

Signature: two.as_integer_ratio()
Docstring:
Return a pair of integers, whose ratio is exactly equal to the original float.

The ratio is in lowest terms and has a positive denominator.  Raise
OverflowError on infinities and a ValueError on NaNs.

>>> (10.0).as_integer_ratio()
(10, 1)
>>> (0.0).as_integer_ratio()
(0, 1)
>>> (-.25).as_integer_ratio()
(-1, 4)
Type:      builtin_function_or_method

In [119]:
two.as_integer_ratio()

(2, 1)

In [120]:
one_eighth = 0.125

In [121]:
one_eighth.as_integer_ratio()

(1, 8)

In [122]:
tau.as_integer_ratio()

(884279719003555, 140737488355328)